# Notebook to study short term persistence from multiple exposures in a single visit 


In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sigmaclip
from astropy.wcs import WCS
import os,glob

%matplotlib notebook

In [ ]:
pwd

In [ ]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'

In [ ]:
# conversion factor from days to seconds
daytosec = 24.*3600.

In [ ]:
#Read files header, make sure they are sorted by EXPSTART
#This now copies the files into the mosaic hi res directory, keeping visit structure

sflts= []

for vis in ['1','2','3']:
    qldir = pdir+'/14016_data/Visit0'+vis+'/'
    wdir = mdir+'/Visit0'+vis+'/'
    if not os.path.isdir(wdir):
        os.mkdir(wdir)
    flts = glob.glob(qldir+'*_flt.fits')
    print('***************')
    starttimes = []
    endtimes   = []
    imagetypes = []
    for flt in flts:
        starttimes.append(fits.getheader(flt,0)['EXPSTART'])
        endtimes.append(fits.getheader(flt,0)['EXPEND'])
        imagetypes.append(fits.getheader(flt,0)['IMAGETYP'])
        filename = os.path.split(flt)[-1]
        if not os.path.exists(wdir+filename):
            shutil.copy(flt, wdir)
            shutil.copy(flt.replace('_flt','_ima'), wdir)
            
    flts = glob.glob(wdir+'*_flt.fits')    
    ii = np.argsort(starttimes)
    for jj in range(len(flts)):
        print(starttimes[ii[jj]],endtimes[ii[jj]],(-starttimes[ii[jj]]+endtimes[ii[jj]])*daytosec,imagetypes[ii[jj]],flts[ii[jj]][-18:])

    sflts.append([flts[i] for i in ii])

In [ ]:
# Choose which visit to work on

from matplotlib.path import Path
import matplotlib.patches as patches


plt.style.use('bmh')

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 19
plt.rcParams['ytick.labelsize'] = 19
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 21
#axarr[j,k].set_axis_bgcolor('#FFFFFF')

visit_index = 2
vsflts = sflts[visit_index]

vsflts = vsflts[0:7]

#plot all exposures multiple times for visualization of the selected pixels

sf=1.5
fig = plt.figure(figsize=(sf*len(vsflts),sf*len(vsflts)))

name = ['Cluster (dith. 1)','Cluster (dith. 2)','Cluster (dith. 3)','Dark 1','Dark 2','Dark 3','Dark 4']

ax = []
for i,flt in enumerate(vsflts):
    im = fits.getdata(flt)
    c, low, upp = sigmaclip(im, 2.,2.)
    mn = np.mean(c)
    print(flt,'clipped mean: ',mn, 'AD sky:')
    j = -1
    while j < i: 
        j+=1
        ax.append(plt.subplot(len(vsflts),len(vsflts),j*len(vsflts)+1+i))
            
        ax[-1].imshow(np.log10(im/mn),cmap='viridis', interpolation='none', origin='lower')
        #ax[-1].set_title(flt[-18:-9],fontsize=6)

        ax[-1].set_title(name[i],fontsize=13)
        
        ax[-1].get_xaxis().set_ticks([])
        ax[-1].get_yaxis().set_ticks([])
        
        ax[-1].set_axis_bgcolor('#FFFFFF')

        if j == i:
            dst = .15
            autoAxis = ax[-1].axis()
            rec = patches.Rectangle((autoAxis[0]-dst,autoAxis[2]-dst),(autoAxis[1]-autoAxis[0])+2*dst,(autoAxis[3]-autoAxis[2])+2*dst,fill=False,lw=3,color='orange')
            rec = ax[-1].add_patch(rec)
            rec.set_clip_on(False)
        else:
            dst = .02
            autoAxis = ax[-1].axis()
            rec = patches.Rectangle((autoAxis[0]-dst,autoAxis[2]-dst),(autoAxis[1]-autoAxis[0])+2*dst,(autoAxis[3]-autoAxis[2])+2*dst,fill=False,lw=1,color='black')
            rec = ax[-1].add_patch(rec)
            rec.set_clip_on(False)

            
# Read the mosaic file and plot it

plt.tight_layout(pad=.225)
mosaic = fits.open(mdir+'/F140W_Mosaic_WFC3_IR_drz.fits')

ax.append(plt.subplot(len(vsflts),len(vsflts),j*len(vsflts)+1))

pos1 = ax[-1].get_position() # get the original position 
pos2 = [pos1.x0, pos1.y0,  pos1.width * len(vsflts)/2.2, pos1.height * len(vsflts)/2.2] 
ax[-1].set_position(pos2) # set a new position
ax[-1].get_xaxis().set_ticks([])
ax[-1].get_yaxis().set_ticks([])
        
c, low, upp = sigmaclip(mosaic[1].data[np.where(np.isfinite(mosaic[1].data))], 2.,2.)
mn = np.mean(c)
im = ax[-1].imshow(np.log10(mosaic[1].data/mn),cmap='viridis', interpolation='none', origin='lower')
ax[-1].set_title('Drizzled mosaic',fontsize=16)

plt.savefig('/Users/gennaro/Congressi_Seminari/SDW_2017/Mosaic.pdf',format='pdf')
plt.savefig('/Users/gennaro/Congressi_Seminari/SDW_2017/Mosaic.png',dpi=300)


In [ ]:
# Choose which visit to work on

from matplotlib.path import Path
import matplotlib.patches as patches


plt.style.use('bmh')

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 19
plt.rcParams['ytick.labelsize'] = 19
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 21
#axarr[j,k].set_axis_bgcolor('#FFFFFF')

visit_index = 2
vsflts = sflts[visit_index]

vsflts = vsflts[0:7]

#plot all exposures multiple times for visualization of the selected pixels

sf=3
fig = plt.figure(figsize=(16,8))

name = ['Cluster (dith. 1)','Cluster (dith. 2)','Cluster (dith. 3)','Dark 1','Dark 2','Dark 3','Dark 4','Dark 5']

ax = []
for i,flt in enumerate(vsflts):
    
    im = fits.getdata(flt)
    c, low, upp = sigmaclip(im, 2.5,2.5)
    mn = np.mean(c)
    print(flt,'clipped mean: ',mn, 'AD sky:')
    j = -1
    while j < 2:
        if j < i:
            j+=1
            ax.append(plt.subplot(3,len(vsflts),j*len(vsflts)+1+i))

            if (i < 3):
                ax[-1].imshow(np.log10(im/mn),cmap='viridis', interpolation='none', origin='lower',clim=(-.666,1.666))
            else:
                ax[-1].imshow(np.log10((im+0.04)/0.08),cmap='viridis', interpolation='none', origin='lower',clim=(-.8,1.25))

            ax[-1].set_title(name[i],fontsize=15)

            ax[-1].get_xaxis().set_ticks([])
            ax[-1].get_yaxis().set_ticks([])

            ax[-1].set_axis_bgcolor('#FFFFFF')

            if j == i:
                dst = .2
                autoAxis = ax[-1].axis()
                rec = patches.Rectangle((autoAxis[0]-dst,autoAxis[2]-dst),(autoAxis[1]-autoAxis[0])+2*dst,(autoAxis[3]-autoAxis[2])+2*dst,fill=False,lw=4,color='orange')
                rec = ax[-1].add_patch(rec)
                rec.set_clip_on(False)
            else:
                dst = .03
                autoAxis = ax[-1].axis()
                rec = patches.Rectangle((autoAxis[0]-dst,autoAxis[2]-dst),(autoAxis[1]-autoAxis[0])+2*dst,(autoAxis[3]-autoAxis[2])+2*dst,fill=False,lw=2,color='black')
                rec = ax[-1].add_patch(rec)
                rec.set_clip_on(False)
        else:
            j+=1
            
plt.tight_layout()
plt.savefig('/user/gennaro/Functional_work/ISRs/Short_term_persistence_I/Figures/Mosaic.pdf')
